In [3]:
from cugrad.tensor import Tensor
a = Tensor([1]); a.data = [2.0]; a.label = 'a'
b = Tensor([1]); b.data = [3.0]; b.label = 'b'
c = Tensor([1]); c.data = [1.0]; c.label = 'c'
d = a * b; d.label = 'd'
e = d + c; e.label = 'e'
e.backward()

def dfs(t):
    print(t)
    for child in t.children:
        dfs(child)

dfs(e)

In [8]:
from cugrad.tensor import Tensor
from cugrad.nn import Layer
from draw_graph import draw_compute_graph

def main():
    # Create tensors
    a = Tensor([1]); a.data = [2.0]
    a.label = 'a'
    b = Tensor([1]); b.data = [3.0]
    b.label = 'b'
    c = Tensor([1]); c.data = [1.0]
    c.label = 'c'
    
    # Perform operations
    d = a * b
    d.label = 'd'
    e = d + c
    e.label = 'e'
    
    # Perform backward pass
    e.backward()
    
    # Print tensors
    print(e)
    print(d)
    print(a)
    print(b)
    print(c)
    
    # Visualize the computation graph
    draw_compute_graph(e, filename="example_compute_graph", format="png")
    
    # Example usage of Layer
    x0 = Tensor([1]); x0.data = [2.0];
    x0.label = 'x0'
    x1 = Tensor([1]); x1.data = [3.0];
    x1.label = 'x1'
    x = Tensor([2]); x.data = [x1.data[0], x0.data[0]];
    n = Layer(2, 3)
    
    res = n(x)
    print(res)
    
    # Perform backward pass on one of the results
    res.backward()
    
    # Visualize the computation graph for the last result
    draw_compute_graph(res, filename="layer_compute_graph", format="png")

if __name__ == "__main__":
    main()


Computation graph saved as example_compute_graph.png
Computation graph saved as layer_compute_graph.png
